In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_30/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# <h1><center><ins>GRIDSEARCH</ins></center></h1>
<h1><center>Checkpoint —solución—:</center></h1>
<img src="img/01_gs.jpg" alt="Drawing" style="width: 300px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#section_objetivos'>0. Objetivos de la Notebook</a>


### <a href='#section_repaso'>1. Lo que tenés que hacer es...</a>
- #### <a href='#section_reg_vs_clas'>1.1 ...cargar el dataset y preparar los datos para el modelo: `KNeighborsRegressor`</a>
- #### <a href='#1.2'>1.2 ...correr GridSearchCV:</a>
- #### <a href='#1.3'>1.3 ...ahora con RandomizedSearchCV:</a>

<a id="section_objetivos"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

<a href='#tabla_contenidos'>Volver a TOC</a>

***

<a id="section_repaso"></a> 
## 1. Lo que tenés que hacer es...

<a id="section_reg_vs_clas"></a>
### 1.1 ...cargar el dataset y preparar los datos para el modelo [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html):

Vamos a volver a utilizar el dataset de la práctica guiada que tenía todos los jugadores de fútbol argentino que están en el juego fifa-18, y contaremos con las mismas variables, salvo que **puntaje_global** va a ser una variable numérica y no categorial: 
- **ID**: un número único que identifica al jugador en toda la base.
- **full_name**: nombre completo del jugador.
- **age**
- **club**: del jugador
- **height_cm**
- **weight_kg**
- **puntaje_global**: puntaje que identifica la habilidad general del jugador.
- **potential**: potencia física del jugador.
- **ritmo**: velocidad de aceleración del jugador.
- **disparos**: nivel de precisión y potencia de sus remates.
- **pases**: nivel de precisión en sus pases.
- **amagues**: nivel de habilidad para amagar a un rival.
- **defensa**: capacidad defensiva general del jugador.
- **físico**: estado físico del jugador (nos indicaría qué tan rápido se cansa)

El **objetivo** del dataset es utilizar distintas features vinculadas al jugador para poder predecir qué nivel global tiene.

Tal como hicimos en la práctica guiada, vamos a probar un modelo de vecinos cercanos pero en lugar de usarlo para clasificar, lo vamos a utilizar para una regresión: [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)). Y nuevamente, la idea es buscar cuáles es la combinación de hiperparámetros (`n_neighbors` y `weights`) que mejor predicen el nivel global del jugador. 


**CUIDADO:** revisá en la notebook de la **Práctica Guiada** todos los pasos previos que hicimos antes de instanciar y entrenar el modelo.

In [2]:
# Importá las librerías que vas a necesitar...
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Cargá en un dataframe el data set...
df=pd.read_csv('../Data/fifa_18_jugadores_argentinos_dos.csv',index_col='Unnamed: 0')
df.sample(5)

,ID,full_name,age,club,height_cm,weight_kg,puntaje_global,potential,ritmo,disparos,pases,amagues,defensa,físico
7178,223615,Christian Chimino,29,Club Atlético Huracán,178.0,72.0,68,68,77,46,55,62,63,68
5174,223335,Lucas Diarte,24,Estudiantes de La Plata,175.0,73.0,70,76,64,30,47,61,71,67
14291,238036,Juan Bautista Cejas,19,Estudiantes de La Plata,175.0,70.0,61,75,74,54,58,62,43,54
536,215259,Emiliano Rigoni,24,Zenit St. Petersburg,178.0,74.0,79,85,86,73,76,79,43,61
11274,226386,Luca Sosa,23,Club Atlético Patronato,175.0,71.0,64,70,55,36,46,47,63,69


In [4]:
# Separamos las X y las Y (puntaje_global)...
cols=['age','height_cm','weight_kg','potential','ritmo','disparos','pases','amagues','defensa','físico']
X = df[cols]
y = df['puntaje_global']

In [5]:
# Hacemos el train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=56)
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(769, 10)

(193, 10)

(769,)

(193,)

In [6]:
# Utilizamos sklearn para estandarizar la matriz de Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.2"></a>
### 1.2 ...correr GridSearchCV:

In [7]:
# instanciamos el modelo
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

In [8]:
# armamos los parámetros para GridSearchCV
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']

param_grid = dict(n_neighbors=k_range, weights=weight_options)
print(param_grid)

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'weights': ['uniform', 'distance']}


In [9]:
# importamos e instanciamos el GridSearchCV
from sklearn.model_selection import GridSearchCV
folds=StratifiedKFold(n_splits=10, random_state=19, shuffle=True)
grid = GridSearchCV(knn, param_grid, cv=folds)

In [10]:
# entrenamos el modelo
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=19, shuffle=True),
             estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30],
                         'weights': ['uniform', 'distance']})

In [11]:
# ¿cuál fue la mejor combinación de hiperparámetros encontrada por GridSearchCV? ¿cuál fue su score de validación?
display(grid.best_estimator_,grid.best_score_,grid.best_params_)

KNeighborsRegressor(n_neighbors=6, weights='distance')

0.903792838548202

{'n_neighbors': 6, 'weights': 'distance'}

In [12]:
# ¿y cómo le fue al mejor modelo con los datos de test?
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

y_pred_grid = grid.predict(X_test)

print('r2 = {}'.format(r2_score(y_test, y_pred_grid)))
print('MAE = {}'.format(mean_absolute_error(y_test, y_pred_grid)))
print('RMSD = {}'.format(np.sqrt(mean_squared_error(y_test, y_pred_grid))))


r2 = 0.9030456906831011
MAE = 1.5066592457651484
RMSD = 1.9618097439280648


<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.3"></a>
### 1.3 ...ahora con RandomizedSearchCV::

In [13]:
# usando la misma grilla de parámetros, probá elegir los hiperparámetros con RandomizedSearchCV y evaluarlos
from sklearn.model_selection import RandomizedSearchCV

knn = KNeighborsRegressor()
grid = RandomizedSearchCV(knn, param_grid, cv=folds)

random = RandomizedSearchCV(knn, param_grid, n_iter=20, cv=folds, random_state=10)
random.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=19, shuffle=True),
                   estimator=KNeighborsRegressor(), n_iter=20,
                   param_distributions={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29, 30],
                                        'weights': ['uniform', 'distance']},
                   random_state=10)

In [14]:
print(random.best_estimator_)
print(random.best_score_)
print(random.best_params_)

KNeighborsRegressor(n_neighbors=4, weights='distance')
0.9020993009399481
{'weights': 'distance', 'n_neighbors': 4}


In [15]:
y_pred_grid_random = random.predict(X_test)

print('r2 = {}'.format(r2_score(y_test, y_pred_grid_random)))
print('MAE = {}'.format(mean_absolute_error(y_test, y_pred_grid_random)))
print('RMSD = {}'.format(np.sqrt(mean_squared_error(y_test, y_pred_grid_random))))

r2 = 0.8927512419754463
MAE = 1.5722379265117237
RMSD = 2.0633336674552067


<a href='#tabla_contenidos'>Volver a TOC</a>